In [1]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets


In [2]:
def relu(x):
    if x < 0:
        return 0
    else:
        return x

In [3]:
def make_exp_rise_input(num_steps, dt, stim_onset, base_current, new_current, tau_rise):
    """
    Returns an array of input current:
    - base_current before stim_onset
    - rises exponentially to new_current after stim_onset and stays there
    """
    input_array = np.full(num_steps, base_current)
    onset_idx = int(stim_onset / dt)
    # print("new current: ", new_current)
    if onset_idx < num_steps:
        t = np.arange(num_steps - onset_idx) * dt
        rise = (new_current - base_current) * (1 - np.exp(-t / tau_rise))
        input_array[onset_idx:] = base_current + rise
    return input_array


In [4]:
def dynamics(w_ll, w_le, w_li, w_el, w_ee, w_ei, w_il, w_ie, w_ii, bg_l, bg_e, bg_i):

    duration = 30 # seconds
    dt = 10e-4
    num_steps = int(duration/dt)

    r_l = np.zeros(num_steps)
    r_e = np.zeros(num_steps)
    r_i = np.zeros(num_steps)

    stim_array = np.zeros(num_steps)
    stim_on_idx = int(5/dt)
    stim_off_idx = int(15/dt)
    # stim_array[stim_on_idx:stim_off_idx] = 1 # stimulus turns on at some point

    time_array = np.arange(0, num_steps, dt)

    tau_e = 2e-3
    tau_l = 2e-3
    tau_i = 2e-3

    stim_onset = 10
    # print(f"stim_onset: {stim_onset}")
    tau_rise = 5e-1

    input_l = make_exp_rise_input(num_steps, dt, stim_onset, bg_l, new_current=2*bg_l, tau_rise=tau_rise)
    input_e = make_exp_rise_input(num_steps, dt, stim_onset, bg_e, new_current=2*bg_e, tau_rise=tau_rise)
    input_i = make_exp_rise_input(num_steps, dt, stim_onset, bg_i, new_current=2*bg_i, tau_rise=tau_rise)
    # plt.plot(input_l)
    for t in range(1, num_steps):

        dr_e = -r_e[t-1] + relu(w_el*r_l[t-1] + w_ee*r_e[t-1] - w_ei*r_i[t-1] + bg_e*input_e[t-1])
        dr_l = -r_l[t-1] + relu(w_ll*r_l[t-1] + w_le*r_e[t-1] - w_li*r_i[t-1] + bg_l*input_l[t-1])
        dr_i = -r_i[t-1] + relu(w_il*r_l[t-1] + w_ie*r_e[t-1] - w_ii*r_i[t-1] + bg_i*input_i[t-1])

        r_e[t] = r_e[t-1] + (dt/tau_e)*dr_e
        r_l[t] = r_l[t-1] + (dt/tau_l)*dr_l
        r_i[t] = r_i[t-1] + (dt/tau_i)*dr_i

    return r_l, r_e, r_i

In [ ]:
@widgets.interact(alpha = widgets.FloatSlider(1, min=0, max=3, step=.01),
                  J = widgets.FloatSlider(1, min=0, max=3, step=.01),
                  g = widgets.FloatSlider(0.5, min=0, max=3, step=.01),
                  bg_l = widgets.FloatSlider(1, min=0, max=4, step=.1),
                  bg_e = widgets.FloatSlider(0.5, min=0, max=4, step=.1),
                  bg_i = widgets.FloatSlider(1, min=0, max=4, step=.1))
def aquisition_dynamics(alpha, J, g, bg_l, bg_e, bg_i):

    w_ll = w_el = w_il = w_li = J
    w_ee = w_le = w_ie = w_ei = alpha*J
    w_ii = g

    r_l, r_e, r_i = dynamics(w_ll, w_le, w_li, w_el, w_ee, w_ei, w_il, w_ie, w_ii, bg_l, bg_e, bg_i)

    fig, axs = plt.subplots(3, 1, figsize=(7, 4), dpi=120, sharex=True)
    skip = 1000

    axs[0].plot(r_l[skip:])
    axs[0].set_title("LBN")

    axs[1].plot(r_e[skip:])
    axs[1].set_title("EBN")

    axs[2].plot(r_i[skip:])
    axs[2].set_title("PV")

    fig.suptitle("Acquisition dynamics")
    fig.tight_layout()


interactive(children=(FloatSlider(value=1.0, description='alpha', max=3.0, step=0.01), FloatSlider(value=1.0, …

In [ ]:
@widgets.interact(alpha = widgets.FloatSlider(1, min=0, max=3, step=.01),
                  J = widgets.FloatSlider(1, min=0, max=3, step=.01),
                  g = widgets.FloatSlider(0.5, min=0, max=3, step=.01),
                  bg_l = widgets.FloatSlider(1, min=0, max=4, step=.1),
                  bg_e = widgets.FloatSlider(0.5, min=0, max=4, step=.1),
                  bg_i = widgets.FloatSlider(1, min=0, max=4, step=.1))
def aquisition_dynamics(alpha, J, g, bg_l, bg_e, bg_i):

    w_ll = w_el = w_il = w_li = J
    w_ee = w_le = w_ie = w_ei = alpha*J
    w_ii = g*J

    r_l, r_e, r_i = dynamics(w_ll, w_le, w_li, w_el, w_ee, w_ei, w_il, w_ie, w_ii, bg_l, bg_e, bg_i)

    fig, axs = plt.subplots(3, 1, figsize=(7, 4), dpi=120, sharex=True)
    skip = 1000

    axs[0].plot(r_l[skip:])
    axs[0].set_title("LBN")

    axs[1].plot(r_e[skip:])
    axs[1].set_title("EBN")

    axs[2].plot(r_i[skip:])
    axs[2].set_title("PV")

    fig.suptitle("Acquisition dynamics")
    fig.tight_layout()


interactive(children=(FloatSlider(value=1.0, description='alpha', max=3.0, step=0.01), FloatSlider(value=1.0, …